## 104網站(招募方塊)

In [4]:
import requests as r
from bs4 import BeautifulSoup
URL = """https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=104_sj&ro=0&keyword=%E8%B3%87%E6%96%99%E7%A7%91%E5%AD%B8&order=2&asc=0&page={}"""
article_list=[]

for page in range(1,50):
    res = r.get(URL.format(page))
#     res.encoding = 'urf-8'
    soup = BeautifulSoup(res.text, 'lxml')
    articles = soup.select('div.line_bottom')
#     print type(articles)
    try:
        for article in articles:
            article_dict={}
            article_dict['title'] = article.select_one('a').text
            article_dict['cor_name'] = article.select_one('div.compname_summary').select_one('a').text
            article_dict['requirement'] = article.select_one('div.requirement').text
    #         article_dict['location'] = article.select_one('span[itemprop="addressLocality"]')
            article_dict['location'] = article.select_one('div.area_summary').text.strip()
            article_dict['hot_deg'] = article.select_one('div.candidates_summary').text.strip()
            article_list.append(article_dict)
#             print article_dict
    except Exception as e:
        article_dict['title'] =""
        article_dict['location'] =""

ConnectionError: HTTPSConnectionPool(host='www.104.com.tw', port=443): Max retries exceeded with url: /jobbank/joblist/joblist.cfm?jobsource=104_sj&ro=0&keyword=%E8%B3%87%E6%96%99%E7%A7%91%E5%AD%B8&order=2&asc=0&page=17 (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x000000000778E588>: Failed to establish a new connection: [Errno 10060] \xb3s\xbdu\xb9\xc1\xb8\xd5\xa5\xa2\xb1\xd1\xa1A\xa6]\xac\xb0\xb3s\xbdu\xb9\xef\xb6H\xa6\xb3\xa4@\xacq\xae\xc9\xb6\xa1\xa8\xc3\xa5\xbc\xa5\xbf\xbdT\xa6^\xc0\xb3\xa1A\xa9\xce\xacO\xb3s\xbdu\xab\xd8\xa5\xdf\xa5\xa2\xb1\xd1\xa1A\xa6]\xac\xb0\xb3s\xbdu\xaa\xba\xa5D\xbe\xf7\xb5L\xaak\xa6^\xc0\xb3\xa1C',))

In [2]:
import json
jobjson = json.dumps(article_list, ensure_ascii=False)
with open('job1.json', 'w') as f:
    f.write(jobjson.encode('utf-8'))

## json file to mySQL

In [139]:
import MySQLdb
db = MySQLdb.connect(host="localhost",user="root",db="python",passwd="23784409")
cursor = db.cursor()
sql = """CREATE TABLE data_science_job (
    no MEDIUMINT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(20),
    cor_name VARCHAR(20),
    requirement VARCHAR(40),
    location VARCHAR(10),
    hot_deg VARCHAR(15))"""

cursor.execute(sql)
db.close()

In [141]:
import MySQLdb

db = MySQLdb.connect(host="localhost",user="root",db="python",passwd="23784409")
cursor = db.cursor()
db.set_character_set('utf8')

with open('job1.json', 'rb') as f:
    job_list = json.load(f)
    for job in job_list:
        title = job['title'].encode('utf-8')
        cor_name = job['cor_name'].encode('utf-8')
        requirement = job['requirement'].encode('utf-8')
        location = job['location'].encode('utf-8')
        hot_deg = job['hot_deg'].encode('utf-8')
        sql = "INSERT INTO data_science_job (title,cor_name,requirement,location,hot_deg) VALUES ('{}', '{}', '{}', '{}', '{}')".format(title, cor_name, requirement, location, hot_deg)
#         print sql   

        try:
            cursor.execute(sql)
            db.commit()
        except Exception as e:
            db.rollback()
db.close()